In [18]:
import sys
import os
import utils

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

import mne
import os
from ieeg.viz.mri import plot_subj, subject_to_info, plot_on_average #this used to import plot subj sig and nonsig, and plot electrodes from mri directly, but in order to save aaron's changes, i put them into jim_mri
from src.analysis.vis.jim_mri import plot_subj_sig_and_nonsig, _plot_electrodes, get_sub_dir
from ieeg.io import raw_from_layout, get_data
# from utils import load_sig_chans, get_sig_chans_per_subject, make_sig_electrodes_per_subject_and_roi_dict, channel_names_to_indices
from src.analysis.utils.general_utils import *

import json
import pyvista as pv
pv.global_theme.allow_empty_mesh = True

['/Users/erinburns/Desktop/GlobalLocal', '/opt/anaconda3/envs/ieeg/lib/python313.zip', '/opt/anaconda3/envs/ieeg/lib/python3.13', '/opt/anaconda3/envs/ieeg/lib/python3.13/lib-dynload', '', '/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages', 'C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/', 'C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/', 'C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/', 'C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/']


### plot single subject's chosen electrodes

In [15]:
subject_folder='/Users/erinburns/Library/CloudStorage/Box-Box/ECoG_Recon'
#test = subject_to_info('D65',subjects_dir=subject_folder) 

In [9]:
subj = "D0057"
base_path = r'C:\Users\erinburns\Library\CloudStorage\Box-Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
plot_these_elecs = ['RAI2', 'RAI3', 'RAI4']
plot_these_elecs_save_name = '_'.join(plot_these_elecs)

subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj(subj_without_zeroes, subj_dir=subject_folder, picks=plot_these_elecs, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_{plot_these_elecs_save_name}.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')

/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\erinburns\\Library\\CloudStorage\\Box-Box\\CoganLab\\BIDS-1.1_GlobalLocal\\BIDS\\derivatives\\freqFilt\\figs/D57_RAI2_RAI3_RAI4.png'

In [22]:
#subjects
#no D107A cause it is messing it up for now
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0110', 'D0116', 'D0117', 'D0121']

plot multiple subjects on average brain

In [23]:
def load_sig_and_all_electrode_data_and_remove_zeroes_in_subject_names(subjects, base_path, epochs_root_file, anat_root):
    """
    Loads significant and all electrodes for a list of subjects.

    Parameters
    ----------
    subjects : list
        A list of subject IDs (e.g., 'D0001').
    base_path : str
        The base path to the derivatives directory containing the data.
    epochs_root_file : str
        The root filename for the epochs data to identify sig_chans files.

    Returns
    -------
    all_sig_electrodes : dict
        Maps subject ID (e.g., 'D1') to a list of significant electrode names.
    all_electrodes : dict
        Maps subject ID (e.g., 'D1') to a list of all electrode names.
    """
    all_sig_electrodes = {}
    all_electrodes = {}

    for subj in subjects:
        # Clean up subject ID (e.g., 'D0057' -> 'D57')
        subj_without_zeroes = "D" + subj[1:].lstrip('0')

        # Get subject info, which contains all channel names
        info = subject_to_info(subj_without_zeroes, subjects_dir=anat_root)
        all_subj_chan_names = info.ch_names
        
        # Store all electrode names for the subject
        all_electrodes[subj_without_zeroes] = all_subj_chan_names

        # Load significant channel names for the subject
        # NOTE: This assumes `load_sig_chans` returns a list of channel NAMES.
        # If it returns indices, you'll need to convert them to names here.
        filename = os.path.join(base_path, subj, f'sig_chans_{subj}_{epochs_root_file}.json')
        try:
            sig_chans = load_sig_chans(filename)
            all_sig_electrodes[subj_without_zeroes] = sig_chans
        except FileNotFoundError:
            print(f"Warning: Sig chans file not found for {subj}. Skipping.")
            all_sig_electrodes[subj_without_zeroes] = [] # Add empty list

    return all_sig_electrodes, all_electrodes

def make_indices_plottable(electrodes_dict, anat_root):
    """
    Converts a dictionary of electrodes into a list of subjects and a
    single flat list of numerical indices suitable for plot_on_average.
    
    Parameters
    ----------
    electrodes_dict : dict
        A dictionary mapping subject IDs (str) to lists of electrode names.

    Returns
    -------
    subjects : list
        An ordered list of the subject identifiers.
    plottable_indices : list
        A single list of global indices for all electrodes.
    """
    subjects = []
    plottable_indices = []
    length = 0
    
    for subject, electrodes in electrodes_dict.items():
        info = subject_to_info(subject, subjects_dir=anat_root)
        # Find the index for each electrode and add the offset
        for electrode in electrodes:
            if electrode in info['ch_names']:
                plottable_indices.append(length + info['ch_names'].index(electrode))
            else:
                print(f"Warning: Electrode '{electrode}' not found in {subject}")

        length += len(info['ch_names'])
        subjects.append(subject)
        
    return subjects, plottable_indices

# ===================================================================
# --- Main Script ---
# ===================================================================

# 1. SETUP & CONFIGURATION
HOME = os.path.expanduser("~")
#if os.name == 'nt':
    #LAB_root = os.path.join(HOME, "Box", "CoganLab")
#else:
    #LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box")

LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Define file and path names
# epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"
# epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
# epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
# epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

# epochs_root_files = ["Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False", 
# "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"]

epochs_root_files = ["Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"]

for epochs_root_file in epochs_root_files:
    data_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "freqFilt", "figs")
    save_path = os.path.join(LAB_root, "BIDS-1.1_GlobalLocal", "BIDS", "derivatives", "vis", "figs", f'{epochs_root_file}')
    anat_path = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "ECoG_Recon")

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    # 2. DATA LOADING
    # Use our modular function to load all the data
    allSigElectrodes, allElectrodes = load_sig_and_all_electrode_data_and_remove_zeroes_in_subject_names(subjects, data_path, epochs_root_file, anat_root=anat_path)

    # 3. DATA TRANSFORMATION
    # Use our modular function to prepare the data for plotting
    subjects_for_plot, sig_indices = make_indices_plottable(allSigElectrodes, anat_root=anat_path)
    _, all_indices = make_indices_plottable(allElectrodes, anat_root=anat_path) # subjects list will be the same

    # 4. VISUALIZATION
    # Plot all electrodes first in a neutral color
    fig = plot_on_average(subjects_for_plot, picks=all_indices, subj_dir=anat_path, rm_wm=False, hemi='both', color=(0.8, 0.8, 0.8), size=0.4)

    # Overlay the significant electrodes in a bright color
    fig = plot_on_average(subjects_for_plot, picks=sig_indices, subj_dir=anat_path, rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4, fig=fig)

    fig.save_image(os.path.join(save_path, "sigAndAllElectrodes_for_" + str(len(subjects)) + "_subjects_" + epochs_root_file + ".png"))

    # Close the figure to free up memory
    fig.close()

    # plot just the significant electrodes in a bright color
    fig2 = plot_on_average(subjects_for_plot, picks=sig_indices, subj_dir=anat_path, rm_wm=False, hemi='both', color=(1, 1, 0), size=0.4)

    fig2.save_image(os.path.join(save_path, "sigElectrodes_for_" + str(len(subjects)) + "_subjects_" + epochs_root_file + ".png"))

    # Close the figure to free up memory
    fig2.close()

    # plot just all electrodes in a neutral color
    fig3 = plot_on_average(subjects_for_plot, picks=all_indices, subj_dir=anat_path, rm_wm=False, hemi='both', color=(0.8, 0.8, 0.8), size=0.4)

    fig3.save_image(os.path.join(save_path, "allElectrodes_for_" + str(len(subjects)) + "_subjects_" + epochs_root_file + ".png"))

    # Close the figure to free up memory
    fig3.close()

/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077


/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero

Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103
Loaded significant channels for subject D0110
Loaded significant channels for subject D0116


/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero

Loaded significant channels for subject D0117
Loaded significant channels for subject D0121


/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero

/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero origin) to head transformation
  info.set_montage(montage)
/opt/anaconda3/envs/ieeg/lib/python3.13/site-packages/ieeg/viz/mri.py:796: RuntimeWarning: Fiducial point lpa not found, assuming identity RAS (non-zero

FileNotFoundError: [Errno 2] No such file or directory: '/Users/erinburns/Box/ECoG_Recon/D57/elec_recon/D57_elec_locations_RAS_brainshifted.txt'

In [20]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = '/Users/erinburns/Desktop/GlobalLocal/src/analysis/config'
subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                                filename='subjects_electrodestoROIs_dict.json')

# Calculate the total number of electrodes in the 'default_dict' across subjects
total_electrodes = utils.count_electrodes_across_subjects(subjects_electrodestoROIs_dict, subjects)
print('total elecs:', total_electrodes)

sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
}

rois = list(rois_dict.keys())
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi = utils.make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)


#plot sig per selected rois
plot_on_average(["D22", "D28", "D64"],
                     picks=["D22-LPIF4", "D28-LPIO7", "D64-LAI6"],
                     label_every=1, hemi='lh', rm_wm=False, color='red',
                     size=1)

#leave picks field blank to plot all 
plot_on_average(["D22", "D28", "D64"],
                    picks=["D22-LPIF4", "D28-LPIO7", "D64-LAI6"],
                    label_every=1, hemi='lh', rm_wm=False, color='red',
                    size=1)

AttributeError: module 'utils' has no attribute 'make_or_load_subjects_electrodes_to_ROIs_dict'